In [1]:
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.cross_validation import train_test_split
from pyfm import pylibfm
import pandas as pd
import numpy as np
import time
import datetime
import os
import xgboost as xgb
from utils import raw_data_path,dump_pickle,load_pickle,cal_log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import lightgbm as lgb
from matplotlib import pyplot as plt
import matplotlib as mpl
import seaborn as sns
from tqdm import tqdm
from sklearn.preprocessing import minmax_scale



/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
t0 = time.time()
train_data = load_pickle(path='../data_valid/train_final_onehot.pkl')    
train_Y = train_data['is_trade']
train_data.drop('is_trade', axis=1, inplace=True) 


cv_data = load_pickle(path='../data_valid/valid_final_onehot.pkl')
cv_Y = cv_data['is_trade']
cv_data.drop('is_trade', axis=1, inplace=True) 


test_data = load_pickle(path='../data_valid/test_final_onehot.pkl')
test_Y = test_data['is_trade']
test_data.drop('is_trade', axis=1, inplace=True) 

print('train shap:',train_data.shape)
print('cv shape', cv_data.shape)
print('test shape', test_data.shape)


test_file = 'round1_ijcai_18_test_a_20180301.txt'
test = pd.read_table(raw_data_path + test_file,delim_whitespace=True)
test_id = test.instance_id

drop_cols = ['context_id', 'user_id', 'shop_id', 'item_id','item_brand_id', 'instance_id']

train_data.drop(drop_cols,axis=1,inplace=True)
cv_data.drop(drop_cols,axis=1,inplace=True)
test_data.drop(drop_cols,axis=1,inplace=True)

train_data = pd.concat([train_data, cv_data], axis=0).reset_index(drop=True)
train_Y =  pd.concat([train_Y, cv_Y], axis=0).reset_index(drop=True)

df = pd.concat([train_data, test_data], axis=0).reset_index(drop=True)
X = minmax_scale(df.values)
train_data = df.iloc[:len(train_data), :]
test_data= df.iloc[len(train_data):, :]

train_data = [ {v: k for k, v in dict(zip(i, range(len(i)))).items()}  for i in tqdm(train_data.values[:, :10])]
test_data = [ {v: k for k, v in dict(zip(i, range(len(i)))).items()}  for i in tqdm(test_data.values[:, :10])]

print('start')
v = DictVectorizer()
train_data = v.fit_transform(train_data)
print('train')
test_data = v.transform(test_data)
print('test')

fm = pylibfm.FM(num_factors=50, num_iter=10, verbose=True, task="classification", initial_learning_rate=0.0001, learning_rate_schedule="optimal")

fm.fit(train_data,train_Y)


train shap: (278805, 370)
cv shape (63610, 370)
test shape (57411, 370)


100%|██████████| 57411/57411 [00:00<00:00, 150320.14it/s]


start
train
test
Creating validation dataset of 0.01 of training for adaptive regularization
-- Epoch 1
Training log loss: nan
-- Epoch 2
Training log loss: nan
-- Epoch 3


KeyboardInterrupt: 

In [ ]:
train_data

In [ ]:
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.cross_validation import train_test_split
from pyfm import pylibfm

from sklearn.datasets import make_classification

X, y = make_classification(n_samples=1000,n_features=100, n_clusters_per_class=1)
data = [ {v: k for k, v in dict(zip(i, range(len(i)))).items()}  for i in X]

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.1, random_state=42)

v = DictVectorizer()
X_train = v.fit_transform(X_train)
X_test = v.transform(X_test)

fm = pylibfm.FM(num_factors=50, num_iter=10, verbose=True, task="classification", initial_learning_rate=0.0001, learning_rate_schedule="optimal")

fm.fit(X_train,y_train)


In [7]:
X_train

<900x100 sparse matrix of type '<class 'numpy.float64'>'
	with 90000 stored elements in Compressed Sparse Row format>

In [9]:
X

array([[ 0.52770943,  0.41027511, -1.00857925, ..., -0.61487631,
         0.68592807,  0.96337477],
       [-1.01593181, -1.33901732,  0.56640636, ..., -0.74931004,
        -0.26838766,  0.75300492],
       [-0.23765329, -0.1173272 , -0.53645908, ..., -1.95776431,
         0.83088056,  1.25895347],
       ...,
       [-0.70869261,  0.43685178, -0.78968643, ..., -0.17278409,
        -0.64347372,  1.19003026],
       [-0.01723435,  2.07002903,  0.74066149, ..., -1.30170852,
         0.06403428, -0.46236565],
       [-2.44099018,  0.69927239,  2.63653614, ..., -2.509899  ,
        -1.01678362,  1.18424685]])